# Managed vs. External Tables in Unity Catalog

**Objective:** Understand the fundamental differences between Managed and External Tables in Unity Catalog, specifically regarding data location, lifecycle management, and recovery options like `UNDROP`.

## Key Concepts
1.  **Managed Table:**
    *   Created without a `LOCATION` clause.
    *   Data is stored in the default storage path of the Schema (or Catalog/Metastore).
    *   **Drop Behavior:** Dropping the table deletes both metadata AND data files (eventually).
    *   **Recovery:** Can be restored using `UNDROP TABLE` within 7 days.

2.  **External Table:**
    *   Created with a `LOCATION` clause pointing to an External Location.
    *   Data resides in your specified path.
    *   **Drop Behavior:** Dropping the table deletes ONLY metadata. Data files persist.
    *   **Recovery:** Can be recreated by pointing to the existing data location. `UNDROP` also works for metadata restoration.

---

## 1. Setup
We will use the `dev.bronze` schema for the managed table and `dev_ext.bronze_ext` schema for the external table experiments.

In [ ]:
-- Create a Managed Table in the default storage location
CREATE TABLE IF NOT EXISTS dev.bronze.managed_sales (
    id INT,
    amount DOUBLE,
    item STRING
);

INSERT INTO dev.bronze.managed_sales VALUES (1, 500.0, 'Laptop');

-- Verify Metadata and Location
DESCRIBE EXTENDED dev.bronze.managed_sales;

## 2. External Table Creation
To create an external table, we first need to register the external location (if not done already) and then point the table to a specific folder.

In [ ]:
-- Define the path for the external table
-- Note: Ensure the path exists or you have permission to create it
-- Format: abfss://<container>@<storage>.dfs.core.windows.net/external_data/sales

CREATE TABLE IF NOT EXISTS dev.bronze.external_sales (
    id INT,
    amount DOUBLE,
    item STRING
)
LOCATION 'abfss://data@adbeasewithdata01.dfs.core.windows.net/adb/ext_tables/sales';

INSERT INTO dev.bronze.external_sales VALUES (2, 20.0, 'Mouse');

-- Verify Location - It should point to the custom path
DESCRIBE EXTENDED dev.bronze.external_sales;

## 3. The DROP Behavior & UNDROP Feature

### Experiment 1: Dropping a Managed Table
When we drop a managed table in UC, the table disappears from the UI. However, the data is effectively "soft-deleted" for a retention period (typically 7 days), allowing us to use `UNDROP`.

In [ ]:
-- Drop the managed table
DROP TABLE dev.bronze.managed_sales;

-- Verify it is gone
SHOW TABLES IN dev.bronze;

In [ ]:
-- Recover the table using UNDROP
UNDROP TABLE dev.bronze.managed_sales;

-- Verify data is back
SELECT * FROM dev.bronze.managed_sales;

### Experiment 2: Dropping an External Table
When we drop an external table, the metadata is removed from Unity Catalog. The data files in ADLS/S3 remain untouched.

In [ ]:
-- Drop the external table
DROP TABLE dev.bronze.external_sales;

-- To recover, we can simply create the table again pointing to the same location
-- OR use UNDROP if within retention period to restore metadata history
CREATE TABLE IF NOT EXISTS dev.bronze.recovered_external_sales
LOCATION 'abfss://data@adbeasewithdata01.dfs.core.windows.net/adb/ext_tables/sales';

-- Verify data still exists
SELECT * FROM dev.bronze.recovered_external_sales;

## 4. Summary

| Feature | Managed Table | External Table |
| :--- | :--- | :--- |
| **Storage Location** | Managed by UC (Metastore/Catalog/Schema root) | Specified by User (`LOCATION` clause) |
| **Drop Table** | Deletes Metadata & Data (Soft Delete) | Deletes Metadata Only |
| **Data Management** | UC handles file maintenance | User handles file maintenance |
| **Best Use Case** | Default choice, Intermediate tables, BI | Data ingested from outside, Sharing data with other systems |

## Next Steps
In the next video, we will explore **Volumes** in Unity Catalog, which allow you to manage unstructured data files (non-tabular data like images, PDFs, CSVs) using the same governance model.